<center>
    <h1> Reddit Classificator </h1>
</center>


<br>
<center>
<img src="images/startgame.jpg" width=1000 height=500 /> </center>

 <h2>  Abstract   </h2>


In questo tutorial si vuole costruire un sistema di classificazione real-time che permetta di suddividere i post di reddit in categorie, per poi visualizzarne graficamente i risultati. L'obbiettivo è quello di riuscire a predire l'argomento dei post, sulla base del solo titolo, evitando quindi che l'utente debba cliccarci.

<h2> Data flow  </h2>





<center>
<img src="images/dataFlow.png" width=1000 height=800 /> </center>

 <h2> Data Sources  </h2>


Per ottenere i reddit post è stata utilizzata la libreria <span><a href="https://praw.readthedocs.io/en/stable/">PRAW</a><span> di python.
    <p></p>
 

Una volta configurate le credenziali e scelto un subreddit, la libreria permette di ottenere varie informazioni quali il titolo, lo score, l'autore e così via. 
<p>
Si è scelto di utilizzare uno script python tramite cui ottenere in streaming i post più recenti.

<h2>
    
Data Ingestion

</h2>




<img src="images/logst.png" width=200 height=200 /> 
<p></p>
Logstash è un progetto open-source per l'analisi dei log in tempo reale.  



<h3>
    How it works?
</h3>
<p>Si è utilizzato l'HTTP Input Plugin di Logstash, per ricevere i dati dallo script python</p>
<center>
<img src="images/patrick.png" width=300 height=150 /> </center>




Attraverso il  Grok Filter Plugin, sono stati rimossi alcuni campi, superflui ai fini della classificazione, conservando unicamente il titolo dei post. Infine si è configurato l'output in modo tale da inviare i dati ad un cluster Kafka. 


<h2>Kafka</h2>
<br>
<br>
<img src="images/kafka.png" width=200 height=200 /> 
<p>Apache Kafka è una piattaforma per il data streaming distribuita che permette di pubblicare, sottoscrivere, archiviare ed elaborare flussi di record in tempo reale.</p>


<h2>Usage</h2>
<img src="images/kafkaMeme.png" width=600 height=600 /> 
<p>Utilizzando Logstash come producer, egli invierà i dati al topic <b>reddit_post</b>: kafka si occuperà di conservare i dati e di gestirne la fruizione da parte dei consumer. </p>




<h2>
Data processing e machine learning
<h2>
<br>
<br>



<img src="images/spark.png" width=200 height=200 /> 


Il consumer è Spark, ovvero una piattaforma open source per l’elaborazione di analisi dei dati su larga scala, progettata per essere veloce e generica. 

Per allenare il classificatore si è scelto di utilizzare il dataset fornito da  <span><a href="https://www.kaggle.com/datasets/mswarbrickjones/reddit-selfposts">kaggle</a><span> contenente mille post per ognuna delle trenta classi identificate. Putroppo non è stato possibile selezionare tutte le classi di interesse, ma solo alcune, in quanto limitati dalla tecnologia... .
<br>
<br>
<img src="images/spark_model.png" width=1000 height=600 /> 


Fondamentale è stato l'utilizzo della libreria  <span><a href="https://spark.apache.org/docs/latest/api/python/">pyspark</a><span>, la quale permette di creare e salvare delle pipeline con cui performare il machine learning.
Il modello prodotto ha ottenuto un'accuracy del 70% ed in generale delle buone performance.



    

<center><img src="images/spark-accuracy.png" width=400 height=700 /> </center>


Ottenuto il modello si è passati all'utilizzo di Spark Structured Streaming, che ci permette di lavorare su dati in tempo reale, integrandosi perfettamente con Kafka.
<br>


Ogni dato ricevuto, viene passato al modello, il quale effettua la predizione.
<br>

I dati predetti vengono poi depositati su elasticsearch.

<img src="images/cert.png" width=350 height=600 />


<h2>
Data Visualization
</h2>

<img src="images/kibana.png" width=350 height=300 />


Attraverso Kibana aggreghiamo i dati ottenuti da spark, e creiamo delle dashboard che ci permettono di visualizzare l'andamento dei dati, la predizione e altre informazioni, il tutto in tempo reale. Ecco alcuni esempi.

Count of Category
<br>


<img src="images/kibana_1.png" width=1500 height=1000 />


Most five popular words by Category

<img src="images/kibana_2.png" width=1500 height=1000 />
